# Задание 3.2 - сверточные нейронные сети (CNNs) в PyTorch

Это упражнение мы буде выполнять в Google Colab - https://colab.research.google.com/  
Google Colab позволяет запускать код в notebook в облаке Google, где можно воспользоваться бесплатным GPU!  

Авторы курса благодарят компанию Google и надеятся, что праздник не закончится.

Туториал по настройке Google Colab:  
https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d  
(Keras инсталлировать не нужно, наш notebook сам установит PyTorch)


In [1]:
# Intstall PyTorch and download data
!pip3 install torch torchvision

!wget -c http://ufldl.stanford.edu/housenumbers/train_32x32.mat http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2022-01-31 22:05:26--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Распознаётся ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Подключение к ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 416 Requested Range Not Satisfiable

    Файл уже получен полностью; нет действий.

--2022-01-31 22:05:27--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Повторное использование соединения с ufldl.stanford.edu:80.
HTTP-запрос отправлен. Ожидание ответа... 416 Requested Range Not Satisfiable

    Файл уже получен полностью; нет действий.



In [2]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import transforms
from pytorch_helper import PyTorchHelper

# Загружаем данные

Разделяем данные на training и validation.

На всякий случай для подробностей - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Net

In [3]:
def base_lenet():
    return nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4),    
            nn.Flatten(),
            nn.Linear(64*2*2, 10),
          )

# Подбор гиперпараметров

In [4]:
from itertools import product

In [5]:
_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43,0.44,0.47],
                         std=[0.20,0.20,0.20])                   
])

In [6]:
# First, lets load the dataset
data_train = dset.SVHN('./', transform=_transform)
data_test = dset.SVHN('./', split='test', transform=_transform)

In [7]:
def find_hyperparameters(config, data_train, data_test):
    device = torch.device("cuda:0") # Let's make sure GPU is available!
    # First, lets load the dataset
    # The key hyperparameters we're going to tune are learning speed, annealing rate and regularization
    # We also encourage you to try different optimizers as well
    batch_size = 64
    helper = PyTorchHelper(batch_size,  data_train)

    train_indices, val_indices = helper.split(.2)

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, 
                                           sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                         sampler=val_sampler)
    #===============================================

    Hyperparams = namedtuple("Hyperparams", ['learning_rate', 'anneal_epochs', 'reg'])
    RunResult = namedtuple("RunResult", ['model', 'train_history', 'val_history', 'final_val_accuracy'])

    learning_rates = [1e-1]
    anneal_coeff = 0.2
    anneal_epochs = [5]
    regs = [0.0001]
    optimizers = [optim.SGD]

    batch_size = 64
    epoch_num = config['epoch_num']

    # Record all the runs here
    # Key should be Hyperparams and values should be RunResult
    run_record = {} 

    # Use grid search or random search and record all runs in run_record dictionnary 
    # Important: perform search in logarithmic space!

    # TODO: Your code here!

    best_hyperparams = Hyperparams(None, None, None)
    best_result = RunResult(None, None, None, None)

    for lr, reg, anneal_epoch, optimizer in product(learning_rates, regs, anneal_epochs, optimizers):
        lenet_model =base_lenet() # base_lenet()

        lenet_model.type(torch.cuda.FloatTensor)
        lenet_model.to(device)

        loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)

        optimizer = optimizer(lenet_model.parameters(), lr=lr, weight_decay=reg)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=anneal_epoch, gamma=anneal_coeff)

        params = Hyperparams(lr, anneal_epoch, reg)

        print(f"\nCurrent hyperparams: {params}")

        loss_history, train_history, val_history = helper.train_model('best_lenet',lenet_model, train_loader, val_loader, loss, optimizer, epoch_num, scheduler)

        result = RunResult(lenet_model, train_history, val_history, val_history[-1])
        run_record[params] = result

        if best_result.final_val_accuracy is None or best_result.final_val_accuracy < result.final_val_accuracy:
            best_result = result
            best_hyperparams = params

        print("\nCurrent best validation accuracy: %4.2f, best hyperparams: %s" % (best_result.final_val_accuracy, best_hyperparams))

In [8]:
import random

In [9]:
config = {
    'epoch_num': 2
}

find_hyperparameters(config,data_train, data_test)

In [10]:
import ray
from ray import tune

In [11]:
 analysis = tune.run(
        tune.with_parameters(find_hyperparameters,  data_train=data_train, data_test=data_test),
        num_samples=1,
        config=config,
        resources_per_trial={"cpu": 8, "gpu": 1})

2022-01-31 22:05:33,862	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
2022-01-31 22:05:37,118	WARNING function_runner.py:558 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc
find_hyperparameters_3f178_00000,PENDING,


(pid=3620) 
(pid=3620) Current hyperparams: Hyperparams(learning_rate=0.1, anneal_epochs=5, reg=0.0001)
(pid=3620) Average loss: 1.431351, Train accuracy: 0.520288, Val accuracy: 0.767524
Trial find_hyperparameters_3f178_00000 completed. Last result: 


Trial name,status,loc
find_hyperparameters_3f178_00000,TERMINATED,


Trial name,status,loc
find_hyperparameters_3f178_00000,TERMINATED,


2022-01-31 22:06:20,960	INFO tune.py:561 -- Total run time: 43.84 seconds (43.47 seconds for the tuning loop).


(pid=3620) Average loss: 0.711938, Train accuracy: 0.781712, Val accuracy: 0.796055
(pid=3620) 
(pid=3620) Current best validation accuracy: 0.80, best hyperparams: Hyperparams(learning_rate=0.1, anneal_epochs=5, reg=0.0001)


In [12]:
best_val_accuracy = None
best_hyperparams = None
best_run = None

for hyperparams, run_result in run_record.items():
    if best_val_accuracy is None or best_val_accuracy < run_result.final_val_accuracy:
        best_val_accuracy = run_result.final_val_accuracy
        best_hyperparams = hyperparams
        best_run = run_result
        
print("Best validation accuracy: %4.2f, best hyperparams: %s" % (best_val_accuracy, best_hyperparams))
        

NameError: name 'run_record' is not defined